In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q autogluon 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
import xgboost as xgb
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV as gsc
from sklearn.metrics import classification_report, confusion_matrix
import warnings;warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
#Importing the Dataset
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sample = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
def lineplot(col_x,col_y='Transported'):
    sns.lineplot(x=col_x, y=col_y, data = train)
    plt.show();

def catplot(col_x,col_y='Transported'):
    sns.catplot(x=col_x,y=col_y, kind='bar', aspect = 2, data=train)
    plt.show();
    
def check(col):
    print('Null values:', train[col].isnull().sum())
    print('Total:', train[col].value_counts().sum())
    print('Uniques:',train[col].nunique())
    if(len(train[col].value_counts())<10):
       print(train[col].value_counts())
    
def algorithm(alg,param):
    algo=gsc(alg(),param,cv=5,n_jobs=-1)
    algo.fit(x_train,y_train)
    print('Best Param Perfomance: {} \n'.format(algo.best_params_))
    model = alg(**algo.best_params_)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))

In [ ]:
train #dataset visualization

In [ ]:
check('PassengerId')

In [ ]:
train.drop('PassengerId', axis=1, inplace=True)
train.head()

In [ ]:
train.ffill(inplace=True)

In [ ]:
check('HomePlanet')
catplot('HomePlanet')
lineplot('HomePlanet')

In [ ]:
le = LabelEncoder()
train.HomePlanet = le.fit_transform(train.HomePlanet)

In [ ]:
check('CryoSleep')

In [ ]:
train.CryoSleep.replace({True:1, False: 0}, inplace=True)

In [ ]:
check('Cabin')

In [ ]:
train.drop('Cabin', axis=1, inplace=True)

In [ ]:
check('Destination')

In [ ]:
train.Destination = le.fit_transform(train.Destination)

In [ ]:
check('VIP')

In [ ]:
train.VIP.replace({True:1, False:0}, inplace=True)

In [ ]:
train.drop('Name', axis=1, inplace = True)

In [ ]:
train

In [ ]:
x = train.loc[:,:'VRDeck']
x.head()

In [ ]:
y = train.Transported

In [ ]:
y.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
%%time
svc = LinearSVC
param ={'C': [1], 'max_iter':[10000,100000]}
algorithm(svc,param)

In [ ]:
%%time
sgd=SGDClassifier
param={'max_iter':[100,1000,10000],'n_jobs':[-1],
       'penalty':['l2'],'alpha':[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]}
algorithm(sgd,param)

In [ ]:
%%time
gnb = GaussianNB
param={'var_smoothing': np.logspace(0,-9, num=1000)}
algorithm(gnb,param)

In [ ]:
%%time
lr = LogisticRegression
param = {'C':[0.001,0.001,0.1,1,10,100,1000],'max_iter':[100,1000,10000]}
algorithm(lr,param)

In [ ]:
%%time
rfc = RandomForestClassifier
param={'n_estimators': [100,200,300], 'max_depth' : [2, 5, 10, 20]}
algorithm(rfc,param)

In [ ]:
%%time
param = {
    'max_depth': 5,'eta': 0.3,'objective': 'multi:softprob', 'num_class': 10}  
num_round = 20
dtrain=xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
bst = xgb.train(param,dtrain,num_round)
preds= bst.predict(dtest)
best = np.asarray([np.argmax(line) for line in preds])
best = pd.Series(best)
best.replace({1:True, 0:False}, inplace=True)
print(classification_report(y_test, best))

In [ ]:
test.drop(['PassengerId','Cabin','Name'], axis=1, inplace=True)
test.ffill(inplace=True)
test.HomePlanet = le.fit_transform(test.HomePlanet)
test.CryoSleep.replace({True:1, False:0}, inplace = True)
test.CryoSleep = test.CryoSleep.astype(int)
test.Destination = le.fit_transform(test.Destination)
test.VIP.replace({True:1, False:0}, inplace=True)

In [ ]:
predictor = TabularPredictor(label='Transported',eval_metric='accuracy', path='/autogluon', 
    verbosity=0)
predictor.fit(train, presets='best_quality')

In [ ]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)
predictor.leaderboard(only_pareto_frontier=True, silent=True)

In [ ]:
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test, model=model_for_prediction)

In [ ]:
sample.Transported = predictions
sample.Transported.value_counts()

In [ ]:
sample.to_csv('autogluon.csv', index=False)
